#### 1. The Real Image Dataset

In [66]:
import torch.utils.data as D
from pathlib import Path
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms

In [67]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
from torchvision import transforms

basic_transformations = transforms.Compose(
    [
        transforms.PILToTensor(),
        # transforms.Lambda(crop_my_image),
    ]
)

In [69]:
# Define the root path
def return_root_path():
    """ Return the root path for data saving given the current OS """
    if Path.cwd().as_posix().startswith('/'):
        return Path('/nfs/turbo/med-kayvan-lab/')
    return Path('Z:/')

# Map the name of the transformation to the function
transformation_dict = dict(zip(['basic'], [basic_transformations]))

class RealImage(D.Dataset):
    """
    Dataset for Real X-ray Angiograhy Images
    Input:
        dir_list <list of python string>: include the directory of the image
        file_name <python string>: the name of the csv file that contains the information of the image
        transform <python string>: the name of the transformation
    """
    def __init__(self, dir_list, transform='basic', file_name='image_infor.csv'):
        super(RealImage, self).__init__()
        self.dir_list = dir_list
        self.file_name = file_name
        self.image_infor = self.get_image_infor()
        self.root_path = return_root_path()
        self.transform = transformation_dict[transform]

    def get_image_infor(self):
        for i, dir_ in enumerate(self.dir_list):
            image_infor = pd.read_csv(Path(dir_) / self.file_name)
            if i == 0:
                df_combined = image_infor
            else:
                df_combined = df_combined.append(image_infor)
        return df_combined
    
    def adjust_dir(self, str_dir):
        image_dir = Path(str_dir.replace('+', '/'))
        return self.root_path / image_dir.relative_to("Z:/")

    def __getitem__(self, index):
        str_dir = self.image_infor.iloc[index, 1]
        image_raw = Image.open(self.adjust_dir(str_dir)).convert("L")
        image = self.transform(image_raw)
        return image

    def __len__(self):
        return len(self.image_infor)



In [70]:
umr_dir=Path(r'Z:\Projects\Angiogram\Data\Processed\Zijun\Synthetic\Real_Image\UMR\Full')
ukr_dir=Path(r'Z:\Projects\Angiogram\Data\Processed\Zijun\Synthetic\Real_Image\UKR\Full')
dir_list = [umr_dir, ukr_dir]
file_name = 'image_infor.csv'
real_image = RealImage(dir_list, file_name=file_name)


In [71]:
real_image.image_infor

,index,save_dir
0,1,Z:+Projects+Angiogram+Data+Processed+Zijun+Syn...
1,2,Z:+Projects+Angiogram+Data+Processed+Zijun+Syn...
2,3,Z:+Projects+Angiogram+Data+Processed+Zijun+Syn...
3,4,Z:+Projects+Angiogram+Data+Processed+Zijun+Syn...
4,5,Z:+Projects+Angiogram+Data+Processed+Zijun+Syn...
...,...,...
1168,1169,Z:+Projects+Angiogram+Data+Processed+Zijun+Syn...
1169,1170,Z:+Projects+Angiogram+Data+Processed+Zijun+Syn...
1170,1171,Z:+Projects+Angiogram+Data+Processed+Zijun+Syn...
1171,1172,Z:+Projects+Angiogram+Data+Processed+Zijun+Syn...


In [72]:
str_dir = real_image.image_infor.iloc[0, 1]

In [73]:
image_path = Path(str_dir.replace('+', '/'))

In [74]:
update_path = image_path.relative_to("Z:/")

In [75]:
image_raw = Image.open(image_path).convert("L")

In [76]:
image_tensor = basic_transformations(image_raw)

In [77]:
image_tensor.shape

torch.Size([1, 512, 512])

In [78]:
image_tensor.to

tensor([[[61, 61, 60,  ..., 44, 44, 38],
         [60, 61, 60,  ..., 44, 44, 38],
         [60, 61, 60,  ..., 43, 44, 38],
         ...,
         [51, 51, 52,  ..., 54, 53, 46],
         [49, 50, 52,  ..., 53, 53, 45],
         [50, 50, 51,  ..., 53, 52, 43]]], dtype=torch.uint8)

In [14]:
image_tensor = real_image[1]

In [82]:
import torchvision.transforms as T

In [83]:
open_image_as_tensor = lambda x: T.PILToTensor()(Image.open(x).convert("L"))

In [84]:
image_tensor = open_image_as_tensor(image_path)

In [85]:
image_tensor.shape

torch.Size([1, 512, 512])

In [88]:
image_tensor.type()

'torch.ByteTensor'

In [87]:
image_tensor.float()

tensor([[[61., 61., 60.,  ..., 44., 44., 38.],
         [60., 61., 60.,  ..., 44., 44., 38.],
         [60., 61., 60.,  ..., 43., 44., 38.],
         ...,
         [51., 51., 52.,  ..., 54., 53., 46.],
         [49., 50., 52.,  ..., 53., 53., 45.],
         [50., 50., 51.,  ..., 53., 52., 43.]]])

In [30]:
transformation_dict = dict(zip(['None', 'basic'], [None,basic_transformations]))

#### 2. The Generator input Dataset

In [24]:
import torch

In [62]:
# Define the root path
def return_root_path():
    """ Return the root path for data saving given the current OS """
    if Path.cwd().as_posix().startswith('/'):
        return Path('/nfs/turbo/med-kayvan-lab/')
    return Path('Z:/')

# Obtain all the paths to the images from the csv file
def get_image_infor(dir_list, file_name):
    for i, dir_ in enumerate(dir_list):
        image_infor = pd.read_csv(Path(dir_) / file_name)
        if i == 0:
            df_combined = image_infor
        else:
            df_combined = df_combined.append(image_infor)
    return df_combined

# Map the name of the transformation to the function
transformation_dict = dict(zip(['None', 'basic'], [None, basic_transformations]))

open_image_as_tensor = lambda x: T.PILToTensor()(Image.open(x).convert("L"))

class GeneratorInput(D.Dataset):
    """
    Dataset for Generator Input, containing 
        the real bakcground image, 
        the synthetic volumn,
        and the synthetic mask.
    Input:
        dir_list <list of python string>: include the directory of the images
        file_name <python string>: the name of the csv file that contains the information of the image
        transform <python string>: the name of the transformation
    """

    def __init__(self, dir_list, transform='basic', file_name='stenosis_detail.csv'):
        super(GeneratorInput, self).__init__()
        self.image_infor = get_image_infor(dir_list, file_name)
        self.root_path = return_root_path()
        self.transform = transformation_dict[transform]
        self.folder_col_id = self.image_infor.columns.get_loc('output_folder')

    def adjust_dir(self, str_dir):
        image_dir = Path(str_dir.replace('+', '/'))
        return self.root_path / image_dir

    def __getitem__(self, index):
        str_dir = self.adjust_dir(self.image_infor.iloc[index, self.folder_col_id])    
        image_volumn = open_image_as_tensor(str_dir/'volumn.png')
        image_mask = open_image_as_tensor(str_dir/'mask.png')
        image_background = open_image_as_tensor(str_dir/'background.png')
        images = self.concatenate_image([image_volumn, image_mask, image_background])
        return images

    def __len__(self):
        return len(self.image_infor)
    
    def concatenate_image(self, image_list):
        return torch.cat(image_list, dim=0)

In [63]:
# Retrive the column of dataframes with the given name
def get_column_index_element(df, index, name):
    return df.iloc[index, df.columns.get_loc(name)]


In [64]:
ukr_dir=Path(r'Z:\Projects\Angiogram\Data\Processed\Zijun\Synthetic\GAN_Data\UKR')
umr_dir=Path(r'Z:\Projects\Angiogram\Data\Processed\Zijun\Synthetic\GAN_Data\UoMR')
dir_list = [umr_dir, ukr_dir]
file_name = 'stenosis_detail.csv'
real_image = GeneratorInput(dir_list, file_name=file_name)

In [65]:
images  = real_image[1]

NameError: name 'T' is not defined

In [40]:
images.shape

torch.Size([3, 512, 512])

In [36]:
concatnated_images = torch.cat([real_image[i].unsqueeze(0) for i in range(4)], dim=0)

In [38]:
concatnated_images.shape

torch.Size([4, 3, 512, 512])

In [1]:
from pathlib import Path


In [2]:
umr_dir=Path(r'Z:\Projects\Angiogram\Data\Processed\Zijun\Synthetic\GAN_Data\UoMR')

In [3]:
umr_dir.relative_to(Path('Z:'))

WindowsPath('/Projects/Angiogram/Data/Processed/Zijun/Synthetic/GAN_Data/UoMR')

### The dataloader

In [5]:
import torch.utils.data as module_dataloader

In [55]:
variables = {'batch_size': 4,
    'collate_fn':module_dataloader.dataloader.default_collate,
    'sampler':module_dataloader.sampler.RandomSampler(range(len(real_image))),
    'drop_last':True,
    'shuffle':False}

In [51]:
len(real_image)

2268

In [52]:
real_image[1].shape

torch.Size([1, 512, 512])

In [61]:
real_image[1].float().type()

'torch.FloatTensor'

In [53]:
import torch 

In [56]:
real_image_dataloader = torch.utils.data.DataLoader(real_image, **variables)

In [57]:
len(real_image_dataloader)

567

In [33]:
real_image_dataloader.dataset

In [35]:
dataset1 = torch.tensor([0, 1, 2, 3, 4, 5])
dataset2 = torch.tensor([10, 11, 12, 13, 14, 15, 16, 17,23, 123, 25, 34, 56, 78, 89])

dataloader1 = torch.utils.data.DataLoader(dataset1, **variables)
dataloader2 = torch.utils.data.DataLoader(dataset2, **variables)

In [37]:
dataloader1 = torch.utils.data.DataLoader(dataset1, batch_size=2, shuffle=True, num_workers=3)

In [38]:
len(dataloader1)

3